In [1]:
# set the matplotlib backend so figures can be saved in the background
import matplotlib
matplotlib.use("Agg")
# import the necessary packages
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import VGG16
from keras.layers.core import Dropout
from keras.layers.core import Flatten
from keras.layers.core import Dense
from sklearn.metrics import confusion_matrix
from keras.layers import Input
from keras.models import Model
from keras.utils import multi_gpu_model
from keras.optimizers import SGD
from sklearn.metrics import classification_report
from imutils import paths
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pickle
import os
import tensorflow as tf
#For injection
import keras.backend.tensorflow_backend as tfback

Using TensorFlow backend.


In [2]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"

In [3]:
def _get_available_gpus():
    """Get a list of available gpu devices (formatted as strings).

    # Returns
        A list of available GPU devices.
    """
    #global _LOCAL_DEVICES
    if tfback._LOCAL_DEVICES is None:
        devices = tf.config.list_logical_devices()
        tfback._LOCAL_DEVICES = [x.name for x in devices]
    return [x for x in tfback._LOCAL_DEVICES if 'device:gpu' in x.lower()]

In [4]:
tfback._get_available_gpus = _get_available_gpus
tfback._get_available_gpus()
tf.config.list_logical_devices()

[LogicalDevice(name='/device:CPU:0', device_type='CPU'),
 LogicalDevice(name='/device:GPU:0', device_type='GPU'),
 LogicalDevice(name='/device:GPU:1', device_type='GPU')]

In [5]:
trainPath = '../FuelTypeData/train/'
testPath = '../FuelTypeData/test/'
valPath = '../FuelTypeData/vald'
BATCH_SIZE = 128
PFR_NUM_CLASS = 10
FUEL_TYPE_NUM_CLASS = 5
EPOCHS = 100

In [6]:
totalTrain = len(list(paths.list_images(trainPath)))
totalVal = len(list(paths.list_images(testPath)))
totalTest = len(list(paths.list_images(valPath)))

In [7]:
def  plot_train_history(history,N,path):
    fig, axes = plt.subplots(1, 2, figsize=(20, 5))
    axes[0].plot(history.history['loss'], label='PFR Train Loss')
    axes[0].plot(history.history['val_loss'], label='PFR Val Loss')
    axes[0].set_xlabel('Epochs')
    axes[0].legend()
    
    axes[1].plot(history.history['accuracy'], label='FT Train accuracy')
    axes[1].plot(history.history['val_accuracy'], label='FT Val accuracy')
    axes[1].set_xlabel('Epochs')
    axes[1].legend()
    
    plt.savefig(path)

## Augmentation

In [8]:
# initialize the training data augmentation object
trainAug = ImageDataGenerator('''
    rotation_range=30,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode="nearest"''')
# initialize the validation/testing data augmentation object (which
# we'll be adding mean subtraction to)
valAug = ImageDataGenerator()
# define the ImageNet mean subtraction (in RGB order) and set the
# the mean subtraction value for each of the data augmentation
# objects
mean = np.array([123.68, 116.779, 103.939], dtype="float32")
trainAug.mean = mean
valAug.mean = mean

## Generator

In [9]:
# initialize the training generator
img_size = 224
trainGen = trainAug.flow_from_directory(
    trainPath,
    class_mode="categorical",
    target_size=(img_size, img_size),
    color_mode="rgb",
    shuffle=False,
    batch_size=BATCH_SIZE)
# initialize the validation generator
valGen = valAug.flow_from_directory(
    valPath,
    class_mode="categorical",
    target_size=(img_size, img_size),
    color_mode="rgb",
    shuffle=False,
    batch_size=BATCH_SIZE)
# initialize the testing generator
testGen = valAug.flow_from_directory(
    testPath,
    class_mode="categorical",
    target_size=(img_size, img_size),
    color_mode="rgb",
    shuffle=False,
    batch_size=BATCH_SIZE)

Found 7487 images belonging to 5 classes.
Found 1597 images belonging to 5 classes.
Found 1597 images belonging to 5 classes.


In [10]:
# load the VGG16 network, ensuring the head FC layer sets are left off
baseModel = VGG16(weights="imagenet", include_top=False,input_tensor=Input(shape=(img_size, img_size, 3)))
# construct the head of the model that will be placed on top of the the base model


In [11]:
#baseModel =  pickle.load(open('../Models/VGG16.sav', 'rb'))

In [12]:
# loop over all layers in the base model and freeze them so they will
# *not* be updated during the first training process
for layer in baseModel.layers:
    layer.trainable = False

In [13]:
headModel = baseModel.output
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(512, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(FUEL_TYPE_NUM_CLASS, activation="softmax")(headModel)
# place the head FC model on top of the base model (this will become
# the actual model we will train)
model = Model(inputs=baseModel.input, outputs=headModel)

In [14]:
# compile our model (this needs to be done after our setting our layers to being non-trainable
print("[INFO] compiling model...")
opt = SGD(lr=1e-4, momentum=0.9)
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])
# train the head of the network for a few epochs (all other layers
# are frozen) -- this will allow the new FC layers to start to become
#initialized with actual "learned" values versus pure random
print("[INFO] training head...")
H = model.fit_generator(
    trainGen,
    steps_per_epoch=totalTrain // BATCH_SIZE,
    validation_data=valGen,
    validation_steps=totalVal // BATCH_SIZE,
    epochs=EPOCHS)
# reset the testing generator and evaluate the network after
# fine-tuning just the network head

[INFO] compiling model...
[INFO] training head...
Epoch 1/100
58/58 [==============================] - 92s 2s/step - loss: 7.3706 - accuracy: 0.5662 - val_loss: 0.0272 - val_accuracy: 0.7936
Epoch 2/100
58/58 [==============================] - 81s 1s/step - loss: 1.4697 - accuracy: 0.7803 - val_loss: 0.0304 - val_accuracy: 0.8244
Epoch 3/100
58/58 [==============================] - 85s 1s/step - loss: 0.4590 - accuracy: 0.8603 - val_loss: 0.0599 - val_accuracy: 0.9156
Epoch 4/100
58/58 [==============================] - 82s 1s/step - loss: 0.2529 - accuracy: 0.9112 - val_loss: 0.0127 - val_accuracy: 0.9265
Epoch 5/100
58/58 [==============================] - 106s 2s/step - loss: 0.1562 - accuracy: 0.9408 - val_loss: 0.0373 - val_accuracy: 0.9101
Epoch 6/100
58/58 [==============================] - 83s 1s/step - loss: 0.1891 - accuracy: 0.9258 - val_loss: 0.1786 - val_accuracy: 0.9074
Epoch 7/100
58/58 [==============================] - 72s 1s/step - loss: 0.1561 - accuracy: 0.9316 - va

58/58 [==============================] - 68s 1s/step - loss: 0.0561 - accuracy: 0.9769 - val_loss: 0.1334 - val_accuracy: 0.9592
Epoch 59/100
58/58 [==============================] - 70s 1s/step - loss: 0.0523 - accuracy: 0.9789 - val_loss: 0.5252 - val_accuracy: 0.9653
Epoch 60/100
58/58 [==============================] - 68s 1s/step - loss: 0.0481 - accuracy: 0.9798 - val_loss: 0.3700 - val_accuracy: 0.9769
Epoch 61/100
58/58 [==============================] - 81s 1s/step - loss: 0.0559 - accuracy: 0.9818 - val_loss: 0.0155 - val_accuracy: 0.9653
Epoch 62/100
58/58 [==============================] - 75s 1s/step - loss: 0.0547 - accuracy: 0.9747 - val_loss: 0.1030 - val_accuracy: 0.9551
Epoch 63/100
58/58 [==============================] - 74s 1s/step - loss: 0.0564 - accuracy: 0.9781 - val_loss: 0.0166 - val_accuracy: 0.9612
Epoch 64/100
58/58 [==============================] - 77s 1s/step - loss: 0.0571 - accuracy: 0.9804 - val_loss: 0.0193 - val_accuracy: 0.9639
Epoch 65/100
58/58 

In [15]:
print("[INFO] evaluating after fine-tuning network head...")
testGen.reset()
predIdxs = model.predict_generator(testGen,
    steps=(totalTest // BATCH_SIZE) + 1)
predIdxsClasses = np.argmax(predIdxs, axis=1)
print(classification_report(testGen.classes, predIdxsClasses,
    target_names=testGen.class_indices.keys()))

[INFO] evaluating after fine-tuning network head...
              precision    recall  f1-score   support

    20H280NG       0.95      0.99      0.97       347
     Ethlyne       0.99      0.93      0.96       288
          F1       0.96      0.70      0.81        61
          F2       1.00      0.99      0.99       156
          NG       0.97      1.00      0.98       745

    accuracy                           0.97      1597
   macro avg       0.97      0.92      0.94      1597
weighted avg       0.97      0.97      0.97      1597



In [16]:
labs = testGen.class_indices.keys()
FTLabels = []
for lab in labs:
    FTLabels.append(lab)
FTLabels

['20H280NG', 'Ethlyne', 'F1', 'F2', 'NG']

In [17]:
cm = confusion_matrix(testGen.classes, predIdxsClasses)
cm = pd.DataFrame(cm)
cm.columns = FTLabels
cm.index = FTLabels
cm

,20H280NG,Ethlyne,F1,F2,NG
20H280NG,344,1,2,0,0
Ethlyne,0,267,0,0,21
F1,17,0,43,0,1
F2,2,0,0,154,0
NG,1,2,0,0,742


In [18]:
WARMUP_PLOT_PATH = '../FTIndv/fttrain.png'
plot_train_history(H, EPOCHS, WARMUP_PLOT_PATH)

In [ ]:
pickle.dump(model, open('../Models/FuelTypeModel/FTModel.sav', 'wb'))